In [1]:
# Code used to flag over-segmentation errors
# A. maximize the number of identified errors
# B. minimize false detection of well-segemented cells 
# 
# The over-segmentation pairs are identified with the following criteria: 
# 1) the two segments have a very high correlation (corr > 0.99) 
# 
# 2) the two segments have to be less than 55 pixels apart(centroid position distance)
# 
# 3) at least one of the segments have to be bigger than 7000 pixels in size
# 
# 4) The two segments have to be touching each other
# 
# 5) if both segments are bigger than 20000 in size, it will be flagged and manually inspected
# 
# 6) for ROIs that have more than two corresponding pairs, the corresponding segments will be ranked by correlation coefficient, and they will be flagged for manual inspection
# 
# #1 and #2 are for initial identification of oversegmentation errors 
# 
# #3, #4, #5 are for eliminating the false detections  

import os, sys
import numpy as np
import pandas as pd
from glob import glob 
from skimage.io import imread, imsave
from os.path import abspath, dirname
from scipy import stats
from scipy.spatial import distance,cKDTree
from skimage.measure import regionprops

In [2]:



r1_dir = "D:\\SWAP\\Vincent\\lt171_FlpO\\gene_new_4tile\\outputs\\r1\\"
r2_dir = "D:\\SWAP\\Vincent\\lt171_FlpO\\gene_new_4tile\\outputs\\r2\\"

# lb_dir = "/LHA3_R3_medium/segmentation/LHA3_R3_medium-c2.tif"
# bad_roi='/test/bad_roi_list.npy'
# spot_dir = '/test/LHA3_R3_medium/spots/'
# out_dir = '/test/LHA3_R3_medium/spots/'
# img_dir='/test/LHA3_R3_medium/stitching/export.n5'


fix_dir = "D:\\SWAP\\Vincent\\lt171_FlpO\\gene_new_4tile\\outputs\\r2\\export_sigma3.n5"
reg_dir = "D:\\SWAP\\Vincent\\lt171_FlpO\\gene_new_4tile\\outputs\\r1\\registration\\lt171_gene_4tile_r1-to-lt171_gene_4tile_r2\\warped"  # directory to the registered image
out_dir = "D:\\SWAP\\Vincent\\lt171_FlpO\\gene_new_4tile\\outputs\\r1\\registration\\lt171_gene_4tile_r1-to-lt171_gene_4tile_r2\\warped\\testout-july29"  # where the output files should be saved 
lb_dir  = "D:\\SWAP\\Vincent\\lt171_FlpO\\gene_new_4tile\\outputs\\r2\\segmentation\\lt171_gene_4tile_r2-c3.tif"  # directory to the segmentation mask (tif format accepted here)

bad_roi = os.path.join(out_dir, "bad_roi_list.npy")
spot_dir = os.path.join(r2_dir, "spots")
out_dir = spot_dir
img_dir = fix_dir

print(bad_roi)
print(spot_dir)


# ----

# input
# spotcount_dir      = sys.argv[1]  # directory to assigned spots per neuron based on airlocalize (csv format)
# roi_dir            = sys.argv[2]  # directory to file containing the ROI metadata (neuron volume, etc.)
# GeneName_dir       = sys.argv[3]  # directory to file containing marker gene metadata (gene name, corresponding imaging round and image channel)
# spot_dir           = sys.argv[4]  # directory to folder of airlocalize output (1 file/gene, txt format)
# intensity_dir      = sys.argv[5]  # directory to folder of intensity measurement output (1 file/gene, csv format)
# output_dir         = sys.argv[6]  # directory where output should be stored
output_dir = "D:\\SWAP\\Vincent\\lt171_FlpO\\gene_new_4tile\\outputs\\"

spotcount_dir = os.path.join(spot_dir, 'spots.csv')
roi_dir = os.path.join(spot_dir, 'roi.csv')
GeneName_dir = os.path.join(output_dir, 'GeneName.csv')
# spot_dir = ""
intensity_dir = os.path.join(r2_dir, 'intensities')

D:\SWAP\Vincent\lt171_FlpO\gene_new_4tile\outputs\r1\registration\lt171_gene_4tile_r1-to-lt171_gene_4tile_r2\warped\testout-july29\bad_roi_list.npy
D:\SWAP\Vincent\lt171_FlpO\gene_new_4tile\outputs\r2\spots


In [3]:
# count_dir      = sys.argv[1]  # directory to spot count/neuron (csv format)
# roi_dir        = sys.argv[2]  # directory to neuron metadata (csv format)
# lb_dir         = sys.argv[3]  # directory to segmentation mask (tif format)
# out_dir        = sys.argv[4]  # directory to save output


count_dir = os.path.join(output_dir, "spotcount_dense_spot_corrected.csv")  # directory to spot count/neuron (csv format)
roi_dir = roi_dir
lb_dir = lb_dir
out_dir = output_dir

In [4]:
%%time
df=pd.read_csv(count_dir,sep=',', index_col=0)
roi=pd.read_csv(roi_dir,sep=',',index_col=0).set_index('roi')
df = df.reindex(roi.index)

###Get correlation matrix
corr_raw =df.T.corr()
s_raw = corr_raw.stack()
ii_raw = s_raw[np.logical_and(s_raw > 0.998, s_raw<1.0)].index.tolist()
test=np.asarray(ii_raw)
test.sort(axis=1)
test=np.unique(test,axis=0)

CPU times: total: 15.2 s
Wall time: 15.2 s


In [5]:
df

,spots_c0,spots_c1,spots_c2,spots_c4
roi,,,,
1,21.0,32.0,29.0,14.0
2,6.0,0.0,1.0,0.0
3,10.0,2.0,0.0,0.0
4,3.0,0.0,2.0,0.0
5,12.0,2.0,2.0,1.0
...,...,...,...,...
13754,0.0,0.0,0.0,0.0
13755,4.0,0.0,1.0,0.0
13756,1.0,0.0,0.0,0.0


In [6]:
roi

,z,y,x,area,minor_axis_length,major_axis_length,eccentricity,solidity
roi,,,,,,,,
1,363.489055,17.368291,73.106487,102.469416,8.682837,17.057071,0.509046,0.795172
2,125.553784,51.560521,396.296073,1692.922728,30.310127,42.301903,0.716519,0.836767
3,111.095382,104.351716,254.707325,1133.917848,25.492777,34.220391,0.744959,0.892238
4,55.515114,52.433029,164.453027,1471.898064,27.790082,35.438489,0.784178,0.899034
5,324.728038,91.253685,221.069806,1436.260392,28.336591,35.553770,0.797007,0.887821
...,...,...,...,...,...,...,...,...
13754,206.419432,212.102989,10.116867,234.888696,13.263912,21.561534,0.615166,0.890799
13755,252.961215,197.694189,313.463300,412.632696,15.868981,32.699733,0.485294,0.846490
13756,265.635180,65.784569,393.811699,170.012136,12.697420,20.252999,0.626940,0.702276


In [7]:
(roi['area'] > 7000).sum()

0

In [8]:
roi.loc[11724]

z                    349.752782
y                    226.641684
x                    393.142818
area                 647.876880
minor_axis_length     20.969819
major_axis_length     31.462832
eccentricity           0.666495
solidity               0.892726
Name: 11724, dtype: float64

In [9]:
cand={}
for i in range(0,len(test)):
#     a=roi.loc[float(test[i,0])].to_numpy()[:3]
#     b=roi.loc[float(test[i,1])].to_numpy()[:3]
    
    a=roi.loc[test[i,0]].to_numpy()[:3]
    b=roi.loc[test[i,1]].to_numpy()[:3]
    
    
    dist=distance.euclidean(a,b)
    if dist<55 and dist>0:
#         a_area=roi.loc[float(test[i,0])]['area']
#         b_area=roi.loc[float(test[i,1])]['area']
        
        
        a_area=roi.loc[test[i,0]]['area']
        b_area=roi.loc[test[i,1]]['area']
        
        if np.maximum(a_area,b_area) >7000:
            c=corr_raw.loc[test[i,0],test[i,1]]
            cand[i] = np.append(test[i,:],c)
            cand[i] = np.append(cand[i],dist)
            if np.minimum(a_area,b_area) >20000:
                cand[i]=np.append(cand[i], str('atten'))
            else:
                cand[i]=np.append(cand[i], str('--'))

In [10]:
m=pd.DataFrame.from_dict(data=cand, orient='index')
m

""


In [11]:
cand

{}

In [12]:

m = m.rename(columns={1:'cell_A', 2:'cell_B', 3: 'corr', 4:'dist',5: 'min_size_20000'})
m['cell_A'], m['cell_B'] = np.where(m['cell_A'] > m['cell_B'], [m['cell_B'],m['cell_A'] ], [m['cell_A'] , m['cell_B']])

lb=imread(lb_dir)
lb_stat=regionprops(lb)

KeyError: 'cell_A'

In [ ]:
select={}
for k in range(0,len(m)):
    #a=np.argwhere(lb_4==n.iloc[k]['cell_A'])
    #b=np.argwhere(lb_4==n.iloc[k]['cell_B'])
    id1=m.iloc[k]['cell_A'].copy()
    id2=m.iloc[k]['cell_B'].copy()
    a=lb_stat[int(id1-1)].coords
    b=lb_stat[int(id2-1)].coords
    kdtree_a = cKDTree(a)
    kdtree_b = cKDTree(b)
    #neighbors = kdtree_a.query_ball_tree(kdtree_b, 1)
    nnn=kdtree_a.count_neighbors(kdtree_b,1)
    if nnn>0:
        select[k]=m.iloc[k]
        if nnn<50:
            select[k]=np.append(select[k], str('less_50'))
        else:
            select[k]=np.append(select[k], str('--'))
select=pd.DataFrame.from_dict(data=select, orient='index')
select = select.rename(columns={0:'cell_A', 1:'cell_B', 2: 'corr', 3:'dist',4: 'min_size_20000',5:'touch'})

select.to_csv(os.path.join(out_dir, 'flag_oversegmentation.csv'))
## This output a list of oversegmented ROI pairs, with some flagged for manual inspection. After manual inspection, the list can be used to merge the oversegmented ROI pairs.   